In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("org_test_results_sorted.csv")

In [4]:
df.head()

,name,password,best_pred_pass,avg_edit_dist,min_distance
0,jinkies_0@hotmail.com,jinkies,jinkies,3.246154,0
1,leonid-lebedev00@rambler.ru,leonid,leonid,3.622222,0
2,cheyen1@hotmail.fr,cheyen,cheyen,2.333333,0
3,niknak_niknak@aol.com,niknak,niknak,2.800000,0
4,ali-sel@mail.ru,alisel,alisel,2.929825,0


In [6]:
import re

def get_username_from_email(email):
    match = re.search(r'^([^@]+)@[^@]+$', email)
    if match:
        return match.group(1)
    return None

# Example usage
email = "jinkies_0@hotmail.com"
username = get_username_from_email(email)
print(f'Username: {username}')

Username: jinkies_0


In [8]:
!pip install python-Levenshtein

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 9.5 MB/s eta 0:00:00:00:0100:01


In [9]:
from Levenshtein import distance as lev
lev("123456", "123456")

0

In [10]:
from tqdm.notebook import tqdm
from joblib import Parallel, delayed

tqdm.pandas()

## Finding editing distance between username and password

In [13]:
def avg_and_min_edit_dist(email, actual_pass):
    username = get_username_from_email(email)
    return lev(str(username), str(actual_pass))

results = Parallel(n_jobs=-1, verbose=10)(delayed(avg_and_min_edit_dist)(row['name'], row['password']) for index, row in df.iterrows())
df['user_pass_min_dist'] = pd.DataFrame(results, index=df.index)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0100s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0242s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0225s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0384s.) Setting batch_size=16.
[Parallel(n_jobs=-1)]: Done 188 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 292 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Batch com

In [14]:
df.head()

,name,password,best_pred_pass,avg_edit_dist,min_distance,user_pass_min_dist
0,jinkies_0@hotmail.com,jinkies,jinkies,3.246154,0,2
1,leonid-lebedev00@rambler.ru,leonid,leonid,3.622222,0,10
2,cheyen1@hotmail.fr,cheyen,cheyen,2.333333,0,1
3,niknak_niknak@aol.com,niknak,niknak,2.800000,0,7
4,ali-sel@mail.ru,alisel,alisel,2.929825,0,1


In [15]:
top_100_df = pd.read_csv("data/top_100_pass.csv")

In [16]:
top_100_df.head()

,password,count
0,123456,2034268
1,123456789,1214120
2,qwerty,988396
3,password,738200
4,12345,687856


In [22]:
top_100_pass = top_100_df['password'].tolist()

In [24]:
def avg_and_min_edit_dist(actual_pass):
    edit_distances = [lev(str(actual_pass), str(top_pass)) for top_pass in top_100_pass]
    avg_edit_dist = sum(edit_distances) / len(top_100_pass)
    min_distance = min(edit_distances)
    best_pred_pass = top_100_pass[edit_distances.index(min(edit_distances))]  
    return (best_pred_pass, avg_edit_dist, min_distance)

results = Parallel(n_jobs=-1, verbose=10)(delayed(avg_and_min_edit_dist)(row['password']) for index, row in df.iterrows())
df[['top_100_best_pass', 'top_100_avg_edit_dist', 'top_100_min_distance']] = pd.DataFrame(results, index=df.index)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1801s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0116s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0082s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Done 109 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0121s.) Setting batch_size=16.
[Parallel(n_jobs=-1)]: Done 278 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Batch com

In [25]:
df.head()

,name,password,best_pred_pass,avg_edit_dist,min_distance,user_pass_min_dist,top_100_best_pass,top_100_avg_edit_dist,top_100_min_distance
0,jinkies_0@hotmail.com,jinkies,jinkies,3.246154,0,2,tinkle,7.35,3
1,leonid-lebedev00@rambler.ru,leonid,leonid,3.622222,0,10,19weed,6.90,5
2,cheyen1@hotmail.fr,cheyen,cheyen,2.333333,0,1,)ryan,6.77,4
3,niknak_niknak@aol.com,niknak,niknak,2.800000,0,7,)ryan,7.00,5
4,ali-sel@mail.ru,alisel,alisel,2.929825,0,1,abc123,6.73,5


In [26]:
df.to_csv("data/org_test_results_final.csv", index=False)